In [1]:
USER_FLAG = "--user"

In [2]:
!pip3 install {USER_FLAG} google-cloud-aiplatform==1.7.0 --upgrade
!pip3 install {USER_FLAG} kfp==1.8.9 google-cloud-pipeline-components==0.2.0

In [3]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [1]:
!python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
!python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

KFP SDK version: 1.8.9
google_cloud_pipeline_components version: 0.2.0


In [2]:
import os
PROJECT_ID = ''

if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "mom-seguros"  # @param {type:"string"}

In [25]:
BUCKET_NAME='gs://mom_seguros_images_car'

In [27]:
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip

In [26]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin
REGION="us-east1"

PIPELINE_ROOT = f"{BUCKET_NAME}/pipeline_root/"
PIPELINE_ROOT

env: PATH=/opt/conda/bin:/home/jupyter/.vscode-server/bin/5235c6bb189b60b01b1f49062f4ffa42384f8c91/bin/remote-cli:/home/jupyter/.local/bin:/usr/local/cuda/bin:/opt/conda/bin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.vscode-server/bin/5235c6bb189b60b01b1f49062f4ffa42384f8c91/bin/remote-cli:/home/jupyter/.local/bin:/usr/local/cuda/bin:/opt/conda/bin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin:/home/jupyter/.local/bin


'gs://mom_seguros_images_car/pipeline_root/'

In [28]:
!ls

Dockerfile  component.yaml  data  merged_data.csv  process.ipynb  trainer


In [29]:
BUCKET_NAME

'gs://mom_seguros_images_car'

In [35]:
from kfp.v2 import dsl

@dsl.component
def addition_component(num1: int, num2: int) -> int:
    return num1 + num2


@dsl.pipeline(name='addition-pipeline', pipeline_root=PIPELINE_ROOT)
def my_pipeline(a: int, b: int, c: int = 10):
    add_task_1 = addition_component(num1=a, num2=b)
    add_task_2 = addition_component(num1=add_task_1.output, num2=c)


In [36]:
from kfp.v2 import compiler

compiler.Compiler().compile(pipeline_func=my_pipeline, package_path='custom_train_pipeline.json')

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1266: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


In [37]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [41]:
pipeline_job = aiplatform.PipelineJob(
    display_name="custom-train-pipeline",
    template_path="custom_train_pipeline.json",
    job_id="custom-train-pipeline-{0}".format(TIMESTAMP),
    # parameter_values={
    #     "project_id": PROJECT_ID,
    #     "bucket": BUCKET_NAME,
    # },
    enable_caching=True,
)

In [42]:
pipeline_job.submit()


INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob


InvalidArgument: 400 Some input parameters of the PipelineSpec.root are missing from PipelineJob.runtimeConfig.parameters: ([a, b])